# Floorplan
THIS IS A TEST THING AND PLAYING AROUND! 

There is also a reference to a local data folder - ignore this and change! 

In [1]:
import sys
import time
import os
from os.path import join
os.environ['NUMBA_ENABLE_CUDASIM'] = '1'

import numpy as np
import matplotlib.pyplot as plt
from numba import jit, cuda

Code from simulate.py

In [60]:
def load_data(load_dir, bid):
    SIZE = 512
    u = np.zeros((SIZE + 2, SIZE + 2))
    u[1:-1, 1:-1] = np.load(join(load_dir, f"{bid}_domain.npy"))
    interior_mask = np.load(join(load_dir, f"{bid}_interior.npy"))
    return u, interior_mask


def jacobi(u, interior_mask, max_iter, atol=1e-6):
    u = np.copy(u)

    for i in range(max_iter):
        # Compute average of left, right, up and down neighbors, see eq. (1)
        u_new = 0.25 * (u[1:-1, :-2] + u[1:-1, 2:] + u[:-2, 1:-1] + u[2:, 1:-1])
        u_new_interior = u_new[interior_mask]
        delta = np.abs(u[1:-1, 1:-1][interior_mask] - u_new_interior).max()
        u[1:-1, 1:-1][interior_mask] = u_new_interior
        if delta < atol:
            break
    return u


def summary_stats(u, interior_mask):
    u_interior = u[1:-1, 1:-1][interior_mask]
    mean_temp = u_interior.mean()
    std_temp = u_interior.std()
    pct_above_18 = np.sum(u_interior > 18) / u_interior.size * 100
    pct_below_15 = np.sum(u_interior < 15) / u_interior.size * 100
    return {
        'mean_temp': mean_temp,
        'std_temp': std_temp,
        'pct_above_18': pct_above_18,
        'pct_below_15': pct_below_15,
    }


if __name__ == '__main__':
    # Load data
    LOAD_DIR = 'data'
    with open(join(LOAD_DIR, 'building_ids.txt'), 'r') as f:
        building_ids = f.read().splitlines()

    # if len(sys.argv) < 2:
    #     N = 1
    # else:
    #     N = int(sys.argv[1])
    N = 1
    building_ids = building_ids[:N]

    # Load floor plans
    all_u0 = np.empty((N, 514, 514))
    all_interior_mask = np.empty((N, 512, 512), dtype='bool')
    for i, bid in enumerate(building_ids):
        u0, interior_mask = load_data(LOAD_DIR, bid)
        all_u0[i] = u0
        all_interior_mask[i] = interior_mask

    # Run jacobi iterations for each floor plan
    MAX_ITER = 20_000
    ABS_TOL = 1e-4

    all_u = np.empty_like(all_u0)
    for i, (u0, interior_mask) in enumerate(zip(all_u0, all_interior_mask)):
        u = jacobi(u0, interior_mask, MAX_ITER, ABS_TOL)
        all_u[i] = u

    # Print summary statistics in CSV format
    stat_keys = ['mean_temp', 'std_temp', 'pct_above_18', 'pct_below_15']
    print('building_id, ' + ', '.join(stat_keys))  # CSV header
    for bid, u, interior_mask in zip(building_ids, all_u, all_interior_mask):
        stats = summary_stats(u, interior_mask)
        print(f"{bid},", ", ".join(str(stats[k]) for k in stat_keys))

## 1. & 3.

In [3]:
def process_one_image(id):
    u0, interior_mask = load_data(LOAD_DIR, id)
    u = jacobi(u0, interior_mask, MAX_ITER, ABS_TOL)
    return u0, interior_mask, u

u0_10000, interior_mask_10000, u_10000 = process_one_image('10000')
u0_48792, interior_mask_48792, u_48792 = process_one_image('48792')

In [4]:
plt.figure()
plt.subplot(3, 2, 1)
plt.imshow(u0_10000)
plt.subplot(3, 2, 2)
plt.imshow(u0_48792)

plt.subplot(3, 2, 3)
plt.imshow(interior_mask_10000)
plt.subplot(3, 2, 4)
plt.imshow(interior_mask_48792)

plt.subplot(3, 2, 5)
plt.imshow(u_10000)
plt.subplot(3, 2, 6)
plt.imshow(u_48792)

plt.show()

## 2.

For $N = 15$ it took $217.92$ sec and there are $4571$ in total, so

In [1]:
4571 / 15 * 219.92 / (60 * 60)

18.61582074074074

$18.6$ hours to process all $4571$ floorplans. 

## 4.

Kernprof for `jacobi`

    Timer unit: 1e-06 s

    Total time: 104.39 s
    File: simulate.py
    Function: jacobi at line 14

    Line #      Hits         Time  Per Hit   % Time  Line Contents
    ==============================================================
        14                                           @profile 
        15                                           def jacobi(u, interior_mask, max_iter, atol=1e-6):
        16        10      11929.5   1192.9      0.0      u = np.copy(u)
        17                                           
        18     47282      46112.0      1.0      0.0      for i in range(max_iter):
        19                                                   # Compute average of left, right, up and down neighbors, see eq. (1)
        20     47282   63932586.1   1352.2     61.2          u_new = 0.25 * (u[1:-1, :-2] + u[1:-1, 2:] + u[:-2, 1:-1] + u[2:, 1:-1])
        21     47282   10141530.4    214.5      9.7          u_new_interior = u_new[interior_mask]
        22     47282   20428426.5    432.1     19.6          delta = np.abs(u[1:-1, 1:-1][interior_mask] - u_new_interior).max()
        23     47282    9713563.0    205.4      9.3          u[1:-1, 1:-1][interior_mask] = u_new_interior
        24                                           
        25     47282     116055.2      2.5      0.1          if delta < atol:
        26        10          7.3      0.7      0.0              break
        27        10          3.1      0.3      0.0      return u

    104.39 seconds - simulate.py:14 - jacobi

Seems to be `u_new = 0.25 * (u[1:-1, :-2] + u[1:-1, 2:] + u[:-2, 1:-1] + u[2:, 1:-1])` taking 61.2%.

## 6.

Little confused. 

## 7.

In [6]:
plt.figure()
plt.subplot(1, 3, 1)
plt.imshow(u0_10000)

plt.subplot(1, 3, 2)
plt.imshow(interior_mask_10000)

plt.subplot(1, 3, 3)
plt.imshow(u_10000)

plt.show()

In [7]:
u0_10000.shape, interior_mask_10000.shape, u_10000.shape

In [8]:
@jit(nopython=True)
def jacobi_jit(u, interior_mask, max_iter, atol=1e-6):
    u = np.copy(u)
    for i in range(max_iter):
        # Compute average of left, right, up and down neighbors, see eq. (1)
        u_new = 0.25 * (u[1:-1, :-2] + u[1:-1, 2:] + u[:-2, 1:-1] + u[2:, 1:-1])
        u_new_interior = np.where(interior_mask, u_new, u[1:-1, 1:-1])
        delta = np.abs(u[1:-1, 1:-1] - u_new_interior).max()
        u[1:-1, 1:-1] = u_new_interior
        if delta < atol:
            break
    return u

In [ ]:
""" THIS WORKS
@jit(nopython=True)
def jacobi_jit_for_loop(u, interior_mask, max_iter, atol=1e-6):
    u = np.copy(u)
    
    n, m = u.shape
    u_new = np.copy(u)
    
    n_interior, m_interior = interior_mask.shape
    interior_mask_list = []
    for i in range(n_interior):
        for j in range(m_interior):
            if interior_mask[i, j]:
                interior_mask_list.append((i, j))
    
    # u_new_interior = np.zeros((n-2, m-2))
    u_new_interior = np.copy(u[1:-1, 1:-1])
    delta = 0
    for _ in range(max_iter):
        # for (i, j) in interior_mask_list:
        #     u_new[i, j] = 0.25 * (u[i, j-1] + u[i, j+1] + u[i-1, j] + u[i+1, j])
        
        for i in range(1, n - 1):
            for j in range(1, m - 1):
                u_new[i, j] = 0.25 * (u[i, j-1] + u[i, j+1] + u[i-1, j] + u[i+1, j])

        u_new_small = u_new[1:-1, 1:-1] 
        # u_new_interior = np.where(interior_mask, u_new_small, u[1:-1, 1:-1])
        # u_new_interior = np.copy(u[1:-1, 1:-1])
        
        ## First iteration of removing np.where
        # for i in range(1, n - 1):
        #     for j in range(1, m - 1):
        #         if interior_mask[i, j]:
        #             new_value = u_new_small[i, j]
        #             u_new_interior[i, j] = new_value
        #             # if delta < abs(u[1:-1, 1:-1][i, j] - new_value):
        #             #     delta = abs(u[1:-1, 1:-1][i, j] - new_value)
        #         else:
        #             u_new_interior[i, j] = u[1:-1, 1:-1][i, j]
        
        ## Second iteration of removing np.where
        for (i, j) in interior_mask_list:
            new_value = u_new_small[i, j]
            u_new_interior[i, j] = new_value
                    
        delta = np.abs(u[1:-1, 1:-1] - u_new_interior).max()
        u[1:-1, 1:-1] = u_new_interior
        
        if delta < atol:
            break
    
    return u
"""

In [61]:
# Cleaner version of the code above
@jit(nopython=True)
def jacobi_jit_for_loop(u, interior_mask, max_iter, atol=1e-6):
    u = np.copy(u)
    
    n, m = u.shape
    u_new = np.copy(u)
    
    n_interior, m_interior = interior_mask.shape
    interior_mask_list = []
    for i in range(n_interior):
        for j in range(m_interior):
            if interior_mask[i, j]:
                interior_mask_list.append((i, j))
    
    u_new_interior = np.copy(u[1:-1, 1:-1])
    for _ in range(max_iter):
        for i in range(1, n - 1):
            for j in range(1, m - 1):
                u_new[i, j] = 0.25 * (u[i, j-1] + u[i, j+1] + u[i-1, j] + u[i+1, j])

        u_new_small = u_new[1:-1, 1:-1] 
        for (i, j) in interior_mask_list:
            new_value = u_new_small[i, j]
            u_new_interior[i, j] = new_value

        delta = 0
        for i in range(n - 2):
            for j in range(m - 2):
                if delta < abs(u[1:-1, 1:-1][i, j] - u_new_interior[i, j]):
                    delta = abs(u[1:-1, 1:-1][i, j] - u_new_interior[i, j])
        
        u[1:-1, 1:-1] = u_new_interior
        
        if delta < atol:
            break
    
    return u

In [63]:
# Load data
LOAD_DIR = 'data'
with open(join(LOAD_DIR, 'building_ids.txt'), 'r') as f:
    building_ids = f.read().splitlines()

# if len(sys.argv) < 2:
#     N = 1
# else:
#     N = int(sys.argv[1])
N = 1
building_ids = building_ids[:N]

# Load floor plans
all_u0 = np.empty((N, 514, 514))
all_interior_mask = np.empty((N, 512, 512), dtype='bool')
for i, bid in enumerate(building_ids):
    u0, interior_mask = load_data(LOAD_DIR, bid)
    all_u0[i] = u0
    all_interior_mask[i] = interior_mask

# Run jacobi iterations for each floor plan
MAX_ITER = 20_000
ABS_TOL = 1e-4

all_u = np.empty_like(all_u0)
for i, (u0, interior_mask) in enumerate(zip(all_u0, all_interior_mask)):
    u = jacobi_jit_for_loop(u0, interior_mask, MAX_ITER, ABS_TOL)
    all_u[i] = u

# Print summary statistics in CSV format
stat_keys = ['mean_temp', 'std_temp', 'pct_above_18', 'pct_below_15']
print('building_id, ' + ', '.join(stat_keys))  # CSV header
for bid, u, interior_mask in zip(building_ids, all_u, all_interior_mask):
    stats = summary_stats(u, interior_mask)
    print(f"{bid},", ", ".join(str(stats[k]) for k in stat_keys))

building_id, mean_temp, std_temp, pct_above_18, pct_below_15
10000, 14.01233878811275, 6.367431059312565, 30.941014791508444, 55.542295034537624

From HPC, see batch-job-exercise-7-1.sh

    cat batch-output/python-hpc-02613-project-simulate-exercise-7-1_24735433.err
    Loaded module: cuda/11.8

    real    2m50.909s
    user    2m50.099s
    sys     0m0.189s

    real    7m8.695s
    user    4m34.541s
    sys     2m32.950s

    real    1m25.064s
    user    1m24.420s
    sys     0m0.261s

# 8.

In [64]:
@cuda.jit
def jacobi_kernel(u, u_new, mask):
    j, i = cuda.grid(2)  # j = col, i = row

    if 0 < i < u.shape[0] - 1 and 0 < j < u.shape[1] - 1:
        avg = 0.25 * (u[i, j - 1] + u[i, j + 1] + u[i - 1, j] + u[i + 1, j])

        # TODO: Slides be like: "avoid branching". Me: yOU cAn't tElL mE wHaT tO dO
        # If anyone feels fresh to refactor this to avoid the branching, please do so :))
        if mask[i, j]:
            u_new[i, j] = avg
        else:
            u_new[i, j] = u[i, j]


def jacobi_cuda(u, interior_mask, num_iter):
    u_cutted = np.copy(u[1:-1, 1:-1]) # Cut off the boundary
    d_u = cuda.to_device(u_cutted)
    d_u_new = cuda.device_array_like(u_cutted)
    d_mask = cuda.to_device(interior_mask)

    tpb = (32, 32)
    bpg = ((u_cutted.shape[1] + tpb[1] - 1) // tpb[1],
           (u_cutted.shape[0] + tpb[0] - 1) // tpb[0])

    for _ in range(num_iter):
        jacobi_kernel[bpg, tpb](d_u, d_u_new, d_mask)
        d_u, d_u_new = d_u_new, d_u  

    return np.pad(d_u.copy_to_host(), pad_width=1, mode='constant', constant_values=0)



In [65]:
# Load data
LOAD_DIR = 'data'
with open(join(LOAD_DIR, 'building_ids.txt'), 'r') as f:
    building_ids = f.read().splitlines()

# if len(sys.argv) < 2:
#     N = 1
# else:
#     N = int(sys.argv[1])
N = 1
building_ids = building_ids[:N]

# Load floor plans
all_u0 = np.empty((N, 514, 514))
all_interior_mask = np.empty((N, 512, 512), dtype='bool')
for i, bid in enumerate(building_ids):
    u0, interior_mask = load_data(LOAD_DIR, bid)
    all_u0[i] = u0
    all_interior_mask[i] = interior_mask

# Run jacobi iterations for each floor plan
NUM_ITER = 3_750 
ABS_TOL = 1e-4

all_u = np.empty_like(all_u0)
for i, (u0, interior_mask) in enumerate(zip(all_u0, all_interior_mask)):
    u = jacobi_cuda(u0, interior_mask, NUM_ITER)
    all_u[i] = u

# Print summary statistics in CSV format
stat_keys = ['mean_temp', 'std_temp', 'pct_above_18', 'pct_below_15']
print('building_id, ' + ', '.join(stat_keys))  # CSV header
for bid, u, interior_mask in zip(building_ids, all_u, all_interior_mask):
    stats = summary_stats(u, interior_mask)
    print(f"{bid},", ", ".join(str(stats[k]) for k in stat_keys))

## 9.

We run

    nsys profile -o simulate_9_prof python simulate_9.py 15

and then 

    nsys stats simulate_9_prof

to get

    ** CUDA API Summary (cudaapisum):

    Time (%)  Total Time (ns)  Num Calls   Avg (ns)   Med (ns)   Min (ns)  Max (ns)   StdDev (ns)              Name            
    --------  ---------------  ---------  ----------  ---------  --------  ---------  -----------  ----------------------------
     56.0       9512033752    1620027      5871.5     5812.0      4141    2407293       5166.8  cuLaunchKernel              
     31.7       5388252604     341142     15794.7    15668.0      7189    1644600       3445.2  cudaMemcpyAsync             
      5.8        982373356     170556      5759.8     6154.0      4381    1383004       3514.9  cudaLaunchKernel            
      3.8        650653215     341067      1907.7     1889.0      1227     534970       1263.9  cudaStreamSynchronize       
      1.7        280940898     341097       823.6      814.0       447      24775        224.7  cudaStreamIsCapturing_v10000
      0.8        142747404         10  14274740.4   181066.0     93547  140467444   44340258.3  cudaMalloc                  
      0.1          9434559         24    393106.6   263558.0    182769    1554556     347865.1  cuModuleLoadData            
      0.0          2867385          2   1433692.5  1433692.5   1390269    1477116      61410.1  cudaHostAlloc               
      0.0          2652053         10    265205.3   107915.5     72798     956379     333050.5  cuModuleUnload              
      0.0           284225          1    284225.0   284225.0    284225     284225          0.0  cudaMemGetInfo              
      0.0           226990         59      3847.3     4866.0      1179      14351       2235.2  cudaEventQuery              
      0.0           199294         15     13286.3    10530.0      9756      36415       7129.1  cudaMemsetAsync             
      0.0           194052        768       252.7      209.0       131       3816        208.4  cuGetProcAddress            
      0.0           125589         30      4186.3     3443.0      3109      15568       2763.7  cudaEventRecord             
      0.0            82629         30      2754.3     2334.5      1927      14137       2168.2  cudaEventCreateWithFlags    
      0.0            69036         29      2380.6     2066.0      1835      11287       1719.4  cudaEventDestroy            
      0.0            10964          2      5482.0     5482.0      4794       6170        973.0  cuInit                      
      0.0             2385          3       795.0      238.0       234       1913        968.2  cuModuleGetLoadingMode      

    ** CUDA GPU Kernel Summary (gpukernsum):

    Time (%)  Total Time (ns)  Instances  Avg (ns)  Med (ns)  Min (ns)  Max (ns)  StdDev (ns)     GridXYZ         BlockXYZ                                                     Name                                                
    --------  ---------------  ---------  --------  --------  --------  --------  -----------  --------------  --------------  ----------------------------------------------------------------------------------------------------
     19.5       1903326923     255744    7442.3    7392.0      6848      8800        176.9  2048    1    1   128    1    1  cupy_add__float64_float64_float64                                                                   
     17.3       1686663005     255759    6594.7    6560.0      6112      7680        115.2   512    1    1   512    1    1  cupy_scan_naive                                                                                     
     11.9       1163253978     170511    6822.2    6848.0      6240      8032        159.6  2048    1    1   128    1    1  cupy_getitem_mask                                                                                   
     11.6       1126003790     255759    4402.6    4256.0      3744      5504        265.2   512    1    1   256    1    1  cupy_bsum_shfl                                                                                      
      9.6        933739784     255759    3650.9    3648.0      3264      4831         58.2     1    1    1   512    1    1  cupy_scan_naive                                                                                     
      6.1        591625911      85248    6940.1    6944.0      6816      7424         56.7  2048    1    1   128    1    1  cupy_scatter_update_mask                                                                            
      6.0        584484003      85248    6856.3    6880.0      6527      7424        118.5  2048    1    1   128    1    1  cupy_multiply__float_float64_float64                                                                
      3.2        315059558      85248    3695.8    3680.0      3551      9568         61.9     1    1    1   256    1    1  void cub::CUB_200200_350_370_500_520_600_610_700_750_800_860_890_900_NS::DeviceReduceSingleTileKern…
      2.6        256078783      85248    3003.9    3008.0      2816      9472         98.3     1    1    1     1    1    1  cupy_less__float64_float_bool                                                                       
      1.0         96706941      15835    6107.2    6080.0      5951      7424         87.5    48    1    1   256    1    1  void cub::CUB_200200_350_370_500_520_600_610_700_750_800_860_890_900_NS::DeviceReduceKernel<cub::CU…
      0.8         73284607      14649    5002.7    4992.0      4927      5408         63.2    51    1    1   256    1    1  void cub::CUB_200200_350_370_500_520_600_610_700_750_800_860_890_900_NS::DeviceReduceKernel<cub::CU…
      0.7         71798643      15835    4534.2    4512.0      4479      4928         52.5   768    1    1   128    1    1  cupy_subtract__float64_float64_float64                                                              
      0.7         68746691      15835    4341.4    4320.0      4287      4768         52.4   768    1    1   128    1    1  cupy_absolute__float64_float64                                                                      
      0.5         48005578       8161    5882.3    5888.0      5696      6528        105.1    50    1    1   256    1    1  void cub::CUB_200200_350_370_500_520_600_610_700_750_800_860_890_900_NS::DeviceReduceKernel<cub::CU…
      0.5         43955725       8832    4976.9    4960.0      4512      5408         62.0    45    1    1   256    1    1  void cub::CUB_200200_350_370_500_520_600_610_700_750_800_860_890_900_NS::DeviceReduceKernel<cub::CU…
      0.5         43871973       8746    5016.2    4992.0      4927      6784         67.2    52    1    1   256    1    1  void cub::CUB_200200_350_370_500_520_600_610_700_750_800_860_890_900_NS::DeviceReduceKernel<cub::CU…
      0.4         40494737       8746    4630.1    4609.0      4575      5024         42.0   824    1    1   128    1    1  cupy_subtract__float64_float64_float64                                                              
      0.4         39104565       8832    4427.6    4416.0      4352      4865         59.6   708    1    1   128    1    1  cupy_subtract__float64_float64_float64                                                              
      0.4         38841482       8746    4441.1    4416.0      4383      4831         63.0   824    1    1   128    1    1  cupy_absolute__float64_float64                                                                      
      0.4         37430084       8131    4603.4    4608.0      4543      4992         43.3   808    1    1   128    1    1  cupy_subtract__float64_float64_float64                                                              
      0.4         37426594       8161    4586.0    4576.0      4512      4993         46.4   798    1    1   128    1    1  cupy_subtract__float64_float64_float64                                                              
      0.4         37371377       8832    4231.4    4224.0      4191      4672         47.3   708    1    1   128    1    1  cupy_absolute__float64_float64                                                                      
      0.4         35875344       8131    4412.2    4385.0      4351      4864         62.0   808    1    1   128    1    1  cupy_absolute__float64_float64                                                                      
      0.4         35840003       8161    4391.6    4384.0      4320      4800         57.4   798    1    1   128    1    1  cupy_absolute__float64_float64                                                                      
      0.3         30550739       5493    5561.8    5536.0      5472      7008         60.8    39    1    1   256    1    1  void cub::CUB_200200_350_370_500_520_600_610_700_750_800_860_890_900_NS::DeviceReduceKernel<cub::CU…
      0.3         30046619       6518    4609.8    4608.0      4544      4993         45.6   804    1    1   128    1    1  cupy_subtract__float64_float64_float64                                                              
      0.3         28715363       6518    4405.5    4384.0      4351      4800         61.6   804    1    1   128    1    1  cupy_absolute__float64_float64                                                                      
      0.3         25057662       5194    4824.3    4832.0      4735      7584         64.2    28    1    1   256    1    1  void cub::CUB_200200_350_370_500_520_600_610_700_750_800_860_890_900_NS::DeviceReduceKernel<cub::CU…
      0.2         23397315       5493    4259.5    4256.0      4191      4704         45.5   622    1    1   128    1    1  cupy_subtract__float64_float64_float64                                                              
      0.2         23279023       4735    4916.4    4896.0      4831      5313         58.4    34    1    1   256    1    1  void cub::CUB_200200_350_370_500_520_600_610_700_750_800_860_890_900_NS::DeviceReduceKernel<cub::CU…
      0.2         22345011       5493    4067.9    4064.0      4000      4448         36.9   622    1    1   128    1    1  cupy_absolute__float64_float64                                                                      
      0.2         21797248       4096    5321.6    5312.0      5184      5792         55.7    33    1    1   256    1    1  void cub::CUB_200200_350_370_500_520_600_610_700_750_800_860_890_900_NS::DeviceReduceKernel<cub::CU…
      0.2         21767813       4479    4860.0    4864.0      4799      5440         44.2    23    1    1   256    1    1  void cub::CUB_200200_350_370_500_520_600_610_700_750_800_860_890_900_NS::DeviceReduceKernel<cub::CU…
      0.2         19532517       3493    5591.9    5568.0      5439      6400         62.2    13    1    1   256    1    1  void cub::CUB_200200_350_370_500_520_600_610_700_750_800_860_890_900_NS::DeviceReduceKernel<cub::CU…
      0.2         19339695       4735    4084.4    4064.0      4000      4511         43.6   531    1    1   128    1    1  cupy_subtract__float64_float64_float64                                                              
      0.2         18583657       4735    3924.7    3904.0      3871      4320         44.4   531    1    1   128    1    1  cupy_absolute__float64_float64                                                                      
      0.2         16618637       4479    3710.3    3712.0      3648      4128         50.0   355    1    1   128    1    1  cupy_subtract__float64_float64_float64                                                              
      0.2         16613697       4096    4056.1    4064.0      3999      4449         38.7   524    1    1   128    1    1  cupy_subtract__float64_float64_float64                                                              
      0.2         16067213       4479    3587.2    3584.0      3551      4000         55.0   355    1    1   128    1    1  cupy_absolute__float64_float64                                                                      
      0.2         15964744       4096    3897.6    3904.0      3840      4256         42.7   524    1    1   128    1    1  cupy_absolute__float64_float64                                                                      
      0.1         14037755       3602    3897.2    3903.0      3839      4257         45.8   433    1    1   128    1    1  cupy_subtract__float64_float64_float64                                                              
      0.1         13539958       3602    3759.0    3744.0      3711      4160         60.9   433    1    1   128    1    1  cupy_absolute__float64_float64                                                                      
      0.1         11814125       3493    3382.2    3360.0      3327      3776         45.9   205    1    1   128    1    1  cupy_subtract__float64_float64_float64                                                              
      0.1         11492719       3493    3290.2    3264.0      3231      3648         61.3   205    1    1   128    1    1  cupy_absolute__float64_float64                                                                      
      0.1          8422424       1535    5486.9    5472.0      5376      5888         63.1    27    1    1   256    1    1  void cub::CUB_200200_350_370_500_520_600_610_700_750_800_860_890_900_NS::DeviceReduceKernel<cub::CU…
      0.1          6198738       1592    3893.7    3872.0      3839      4256         46.2   434    1    1   128    1    1  cupy_subtract__float64_float64_float64                                                              
      0.1          5974951       1592    3753.1    3744.0      3680      4096         56.4   434    1    1   128    1    1  cupy_absolute__float64_float64                                                                      
      0.1          5930398       1535    3863.5    3841.0      3807      4224         48.5   426    1    1   128    1    1  cupy_subtract__float64_float64_float64                                                              
      0.1          5732825       1535    3734.7    3712.0      3679      4096         54.0   426    1    1   128    1    1  cupy_absolute__float64_float64                                                                      
      0.0           558618         15   37241.2   38304.0     15680     49184      10907.6     1    1    1   512    1    1  cupy_var_core_float64                                                                               
      0.0           123072         15    8204.8    8288.0      7744      8672        306.2  2065    1    1   128    1    1  cupy_copy__float64_float64                                                                          
      0.0           104543         30    3484.8    3424.0      3360      4032        137.6     1    1    1   512    1    1  cupy_cub_sum_pass2                                                                                  
      0.0           100702         15    6713.5    6624.0      6592      7424        219.5  2048    1    1   128    1    1  cupy_copy__float64_float64                                                                          
      0.0            99227         30    3307.6    3263.0      3104      3905        226.1     1    1    1   256    1    1  void cub::CUB_200200_350_370_500_520_600_610_700_750_800_860_890_900_NS::DeviceReduceSingleTileKern…
      0.0            89915         30    2997.2    2976.0      2943      3328         85.0     1    1    1     1    1    1  cupy_true_divide__int64_int_float64                                                                 
      0.0            89887         30    2996.2    2976.0      2912      3200         73.9     1    1    1     1    1    1  cupy_true_divide__float64_float_float64                                                             
      0.0            88032         30    2934.4    2912.0      2879      3200         74.2     1    1    1     1    1    1  cupy_multiply__float64_float_float64                                                                
      0.0            45023         15    3001.5    2976.0      2944      3328        103.5     1    1    1     1    1    1  cupy_sqrt__float64_float64                                                                          
      0.0            44190         15    2946.0    2912.0      2880      3296        101.5     1    1    1     1    1    1  cupy_copy__float64_float64                                                                          
      0.0            18463          4    4615.8    4576.0      4448      4863        183.1    48    1    1   256    1    1  void cub::CUB_200200_350_370_500_520_600_610_700_750_800_860_890_900_NS::DeviceReduceKernel<cub::CU…
      0.0            16544          4    4136.0    4144.5      3775      4480        348.1    28    1    1   256    1    1  void cub::CUB_200200_350_370_500_520_600_610_700_750_800_860_890_900_NS::DeviceReduceKernel<cub::CU…
      0.0            15904          4    3976.0    3968.0      3936      4032         40.3    51    1    1   256    1    1  void cub::CUB_200200_350_370_500_520_600_610_700_750_800_860_890_900_NS::DeviceReduceKernel<cub::CU…
      0.0            14656          4    3664.0    3616.0      3552      3872        141.9    48    1    1   512    1    1  cupy_cub_sum_pass1                                                                                  
      0.0            14623          4    3655.8    3648.0      3456      3871        214.1    28    1    1   512    1    1  cupy_cub_sum_pass1                                                                                  
      0.0            14497          4    3624.3    3536.0      3521      3904        187.0    51    1    1   512    1    1  cupy_cub_sum_pass1                                                                                  
      0.0             9343          2    4671.5    4671.5      4159      5184        724.8    39    1    1   256    1    1  void cub::CUB_200200_350_370_500_520_600_610_700_750_800_860_890_900_NS::DeviceReduceKernel<cub::CU…
      0.0             9057          2    4528.5    4528.5      4225      4832        429.2    50    1    1   256    1    1  void cub::CUB_200200_350_370_500_520_600_610_700_750_800_860_890_900_NS::DeviceReduceKernel<cub::CU…
      0.0             9056          2    4528.0    4528.0      4224      4832        429.9    52    1    1   256    1    1  void cub::CUB_200200_350_370_500_520_600_610_700_750_800_860_890_900_NS::DeviceReduceKernel<cub::CU…
      0.0             8864          2    4432.0    4432.0      4320      4544        158.4   768    1    1   128    1    1  cupy_greater__float64_float_bool                                                                    
      0.0             8608          2    4304.0    4304.0      4288      4320         22.6   768    1    1   128    1    1  cupy_less__float64_float_bool                                                                       
      0.0             8512          2    4256.0    4256.0      3840      4672        588.3    13    1    1   256    1    1  void cub::CUB_200200_350_370_500_520_600_610_700_750_800_860_890_900_NS::DeviceReduceKernel<cub::CU…
      0.0             8064          2    4032.0    4032.0      3968      4096         90.5    27    1    1   256    1    1  void cub::CUB_200200_350_370_500_520_600_610_700_750_800_860_890_900_NS::DeviceReduceKernel<cub::CU…
      0.0             8000          2    4000.0    4000.0      3872      4128        181.0    33    1    1   256    1    1  void cub::CUB_200200_350_370_500_520_600_610_700_750_800_860_890_900_NS::DeviceReduceKernel<cub::CU…
      0.0             7968          2    3984.0    3984.0      3969      3999         21.2    45    1    1   256    1    1  void cub::CUB_200200_350_370_500_520_600_610_700_750_800_860_890_900_NS::DeviceReduceKernel<cub::CU…
      0.0             7616          2    3808.0    3808.0      3776      3840         45.3    34    1    1   256    1    1  void cub::CUB_200200_350_370_500_520_600_610_700_750_800_860_890_900_NS::DeviceReduceKernel<cub::CU…
      0.0             7552          2    3776.0    3776.0      3712      3840         90.5    23    1    1   256    1    1  void cub::CUB_200200_350_370_500_520_600_610_700_750_800_860_890_900_NS::DeviceReduceKernel<cub::CU…
      0.0             7296          2    3648.0    3648.0      3488      3808        226.3    39    1    1   512    1    1  cupy_cub_sum_pass1                                                                                  
      0.0             7136          2    3568.0    3568.0      3456      3680        158.4    27    1    1   512    1    1  cupy_cub_sum_pass1                                                                                  
      0.0             7136          2    3568.0    3568.0      3552      3584         22.6    50    1    1   512    1    1  cupy_cub_sum_pass1                                                                                  
      0.0             7136          2    3568.0    3568.0      3552      3584         22.6    52    1    1   512    1    1  cupy_cub_sum_pass1                                                                                  
      0.0             7040          2    3520.0    3520.0      3520      3520          0.0    45    1    1   512    1    1  cupy_cub_sum_pass1                                                                                  
      0.0             6976          2    3488.0    3488.0      3455      3521         46.7    13    1    1   512    1    1  cupy_cub_sum_pass1                                                                                  
      0.0             6976          2    3488.0    3488.0      3456      3520         45.3    33    1    1   512    1    1  cupy_cub_sum_pass1                                                                                  
      0.0             6975          2    3487.5    3487.5      3487      3488          0.7    23    1    1   512    1    1  cupy_cub_sum_pass1                                                                                  
      0.0             6944          2    3472.0    3472.0      3456      3488         22.6    34    1    1   512    1    1  cupy_cub_sum_pass1                                                                                  
      0.0             4416          1    4416.0    4416.0      4416      4416          0.0   824    1    1   128    1    1  cupy_greater__float64_float_bool                                                                    
      0.0             4384          1    4384.0    4384.0      4384      4384          0.0   808    1    1   128    1    1  cupy_less__float64_float_bool                                                                       
      0.0             4384          1    4384.0    4384.0      4384      4384          0.0   824    1    1   128    1    1  cupy_less__float64_float_bool                                                                       
      0.0             4384          1    4384.0    4384.0      4384      4384          0.0   804    1    1   128    1    1  cupy_greater__float64_float_bool                                                                    
      0.0             4384          1    4384.0    4384.0      4384      4384          0.0   808    1    1   128    1    1  cupy_greater__float64_float_bool                                                                    
      0.0             4383          1    4383.0    4383.0      4383      4383          0.0   798    1    1   128    1    1  cupy_greater__float64_float_bool                                                                    
      0.0             4352          1    4352.0    4352.0      4352      4352          0.0   798    1    1   128    1    1  cupy_less__float64_float_bool                                                                       
      0.0             4352          1    4352.0    4352.0      4352      4352          0.0   804    1    1   128    1    1  cupy_less__float64_float_bool                                                                       
      0.0             4225          1    4225.0    4225.0      4225      4225          0.0   708    1    1   128    1    1  cupy_greater__float64_float_bool                                                                    
      0.0             4224          1    4224.0    4224.0      4224      4224          0.0   622    1    1   128    1    1  cupy_greater__float64_float_bool                                                                    
      0.0             4192          1    4192.0    4192.0      4192      4192          0.0   708    1    1   128    1    1  cupy_less__float64_float_bool                                                                       
      0.0             4192          1    4192.0    4192.0      4192      4192          0.0   524    1    1   128    1    1  cupy_greater__float64_float_bool                                                                    
      0.0             4031          1    4031.0    4031.0      4031      4031          0.0   622    1    1   128    1    1  cupy_less__float64_float_bool                                                                       
      0.0             3905          1    3905.0    3905.0      3905      3905          0.0   531    1    1   128    1    1  cupy_greater__float64_float_bool                                                                    
      0.0             3873          1    3873.0    3873.0      3873      3873          0.0   531    1    1   128    1    1  cupy_less__float64_float_bool                                                                       
      0.0             3872          1    3872.0    3872.0      3872      3872          0.0   433    1    1   128    1    1  cupy_less__float64_float_bool                                                                       
      0.0             3872          1    3872.0    3872.0      3872      3872          0.0   524    1    1   128    1    1  cupy_less__float64_float_bool                                                                       
      0.0             3841          1    3841.0    3841.0      3841      3841          0.0   433    1    1   128    1    1  cupy_greater__float64_float_bool                                                                    
      0.0             3744          1    3744.0    3744.0      3744      3744          0.0   426    1    1   128    1    1  cupy_greater__float64_float_bool                                                                    
      0.0             3711          1    3711.0    3711.0      3711      3711          0.0   434    1    1   128    1    1  cupy_less__float64_float_bool                                                                       
      0.0             3681          1    3681.0    3681.0      3681      3681          0.0   434    1    1   128    1    1  cupy_greater__float64_float_bool                                                                    
      0.0             3680          1    3680.0    3680.0      3680      3680          0.0   426    1    1   128    1    1  cupy_less__float64_float_bool                                                                       
      0.0             3648          1    3648.0    3648.0      3648      3648          0.0   355    1    1   128    1    1  cupy_greater__float64_float_bool                                                                    
      0.0             3552          1    3552.0    3552.0      3552      3552          0.0   355    1    1   128    1    1  cupy_less__float64_float_bool                                                                       
      0.0             3264          1    3264.0    3264.0      3264      3264          0.0   205    1    1   128    1    1  cupy_less__float64_float_bool                                                                       
      0.0             3264          1    3264.0    3264.0      3264      3264          0.0   205    1    1   128    1    1  cupy_greater__float64_float_bool                                                                    


    ** GPU MemOps Summary (by Time) (gpumemtimesum):

    Time (%)  Total Time (ns)  Count   Avg (ns)  Med (ns)  Min (ns)  Max (ns)  StdDev (ns)      Operation     
    --------  ---------------  ------  --------  --------  --------  --------  -----------  ------------------
        99.3        457609051  341067    1341.7    1313.0      1279      2080         60.2  [CUDA memcpy DtoH]
        0.6          2972864      30   99095.5   98782.5     24000    190046      76270.6  [CUDA memcpy HtoD]
        0.1           363259      45    8072.4    9280.0      3199     13312       3615.4  [CUDA memcpy DtoD]
        0.0            72481      15    4832.1    4896.0      4384      5473        367.6  [CUDA memset]     


    ** GPU MemOps Summary (by Size) (gpumemsizesum):

    Total (MB)  Count   Avg (MB)  Med (MB)  Min (MB)  Max (MB)  StdDev (MB)      Operation     
    ----------  ------  --------  --------  --------  --------  -----------  ------------------
        67.339      45     1.496     2.114     0.262     2.114        0.883  [CUDA memcpy DtoD]
        35.389      30     1.180     1.180     0.262     2.097        0.933  [CUDA memcpy HtoD]
        31.704      15     2.114     2.114     2.114     2.114        0.000  [CUDA memset]     
        1.109  341067     0.000     0.000     0.000     0.000        0.000  [CUDA memcpy DtoH]

## 11.

For $N = 100$ and 32 cores it took $87$ sec so

In [2]:
(4571 / 100) * 87 / (60 * 60)

1.1046583333333333